In [1]:
import nltk
from nltk.corpus import sentiwordnet as swn
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
import requests
import sys, os
import numpy as np
import pandas as pd

Warming up PyWSD (takes ~10 secs)... took 6.164828300476074 secs.


In [2]:
nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

In [3]:
breakdown = swn.senti_synset('good.n.01')
print(breakdown.pos_score())

0.5


In [4]:
wn.synsets('love')

[Synset('love.n.01'),
 Synset('love.n.02'),
 Synset('beloved.n.01'),
 Synset('love.n.04'),
 Synset('love.n.05'),
 Synset('sexual_love.n.02'),
 Synset('love.v.01'),
 Synset('love.v.02'),
 Synset('love.v.03'),
 Synset('sleep_together.v.01')]

In [5]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [12]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'

    review = []
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
            if len(aspect) > 0:
                aspect = aspect[0]
            else:
                aspect = ''
            review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df

In [13]:
def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append((pos['word'], pos['pos']))
    return res

In [14]:
def get_score(sentence, key, tagged_sentence):
    flag = True
    
    tagged = ('','')
    for p in tagged_sentence:
        if p[0] == key:
            tagged = p
    if tagged == ('', ''):
        flag = False

    ambiguous = tagged[0]
    tag = tagged[1]
    pos = ''

    if 'NN' in tag:
        pos = 'n'
    elif 'NNS' in tag:
        pos = 'nns'
    elif 'VB' in tag:
        pos = 'v'
    elif 'VBG' in tag:
        pos = 'v'
    elif 'JJ' in tag:
        pos = 'a'
    elif 'RB' in tag:
        pos = 'r'
    else:
        flag = False

    endscore = 0
    if flag:
        answer = 1#cosine_lesk(sentence, ambiguous, pos)
        if answer:
            try:
                score = swn.senti_synset(ambiguous + '.' + 'a' + '.01')

                if score.pos_score() > score.neg_score():
                    endscore = score.pos_score()
                else:
                    endscore = score.neg_score() * (-1)
            except:
                endscore = 0
    return endscore

In [15]:
negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection"
]


def get_sentence_score(sentence):
    scores = []
    tagged_sentence = pos_tag(sentence)
    for word in sentence.split(' '):
        if word in op_set:
            score = get_score(sentence, word, tagged_sentence)
            scores.append(score)

    if len(scores):
        for neg in negation:
            if neg in sentence:
                return np.mean(scores) * (-1)
        return np.mean(scores)
    return 0

In [16]:
def main_program(review, target, filename):
    scores = []
    labels = []
    index = 0
    for sentence in review:
        aspects = target[index].split(',')
        label = 0
        if aspects[0] != '':
            for aspect in aspects:
                try:
                    tanda = aspect.split('[')[1][0]
                    angka = aspect.split('[')[1][1]
                    if tanda == '+':
                        label += int(angka)
                    else:
                        label -= int(angka)
                except:
                    pass
#                     print(aspect)
            if label != 0:
                score = get_sentence_score(sentence)

                if label > 0:
                    labels.append(1)
                elif label < 0:
                    labels.append(2)

                if score >= 0:
                    scores.append(1)
                elif score < 0:
                    scores.append(2)
            
        index += 1
    
#     data = { 'review':review, 'label': labels, 'prediction': scores}
#     out = pd.DataFrame(data)
#     out.to_csv(filename)
    
    return labels, scores

In [17]:
def preprocessing(sentences):
    res = []
    for sentence in sentences:
        try:
            res.append(sentence.replace("n't", "not").replace("'m", "am"))
        except:
            pass
    return res

In [18]:
filename = [
            'Apex AD2600 Progressive-scan DVD player',
            'Canon G3',
            'Creative Labs Nomad Jukebox Zen Xtra 40GB',
            'Nikon coolpix 4300',
            'Nokia 6610'
]

def run_all():
    out_file = pd.DataFrame(columns=['name', 'TP', 'TN', 'FP', 'FN'])
    for file in filename:
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        
        df = read_file('dataset/bing_liu/' + file + '.txt')
        label, pred = main_program(preprocessing(df['review']), df['target'], 'sentiwordnet.csv')
        for x in range(0, len(label) - 1):
            l = label[x]
            p = pred[x]
            if l == 1 and p == 1:
                TP += 1
            elif l == 2 and p == 2:
                TN += 1
            elif l == 1 and p == 2:
                FP += 1
            elif l == 2 and p == 1:
                FN += 1
        
        out_file = out_file.append({'name': file, 'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN}, ignore_index=True)
        out_file.to_csv("result-sentiwordnet-v3.csv")
        out_file.to_excel("result-sentiwordnet-v3.xlsx")